<a href="https://colab.research.google.com/github/jing-zhou/w2v/blob/master/w2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Environment setup and library import**

In [66]:
import random                           # Generates random numbers
import os                               # Create directories, list files
from zipfile import ZipFile
from shutil import copyfile             # Copy files from Source to Destination
import cv2                              # To flip images in data augmentation
import matplotlib.pyplot as plt         # To save the images
import matplotlib.image as mpimg

**2. Link Goggle Drive with Colab**

Source : https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d 

In [67]:
# Check is Link to Drive is OK
google = !if [ -d 'GDrive/' ]; then echo "1" ; else echo "0"; fi
if (google[0] is '0' ):
  from google.colab import drive
  drive.mount('/content/GDrive/')
!if [ -d 'GDrive/' ]; then echo "Connection to Google drive successful" ; else echo "Error to connect to Google drive"; fi

Connection to Google drive successful


**3. handler function for zip file**

In [68]:
# function for handling zip file, zfile must be FQN
def handleZip(corp, zfile, temp):
  # Create a ZipFile Object and load sample.zip in it
  with ZipFile(zfile, 'r') as zipObj:
    # Extract all the contents of zip file in different directory
    zipObj.extractall(temp)
    # Get a list of all archived file names from the zip
    listOfFileNames = zipObj.namelist()
    # Iterate over the file names
    for fileName in listOfFileNames:
      if fileName.endswith('.zip'):
        handleZip(corp, temp + '/' + fileName, temp)
        continue

      tfile = open(temp + '/' + fileName, 'r')
      corp.write(tfile.read())
      tfile.close()


**4. Create root dataset directory, temp directory and corpus file**


In [69]:
dataset_dir = 'dataset'
if not os.path.exists(dataset_dir):
  os.mkdir(dataset_dir)

corpus_dir = dataset_dir + '/corpus'
if not os.path.exists(corpus_dir):
  os.mkdir(corpus_dir)
corpus_file = '/corpus.txt'
# remove previous file instance
if os.path.exists(corpus_dir + corpus_file):
  os.remove(corpus_dir + corpus_file)

corpus = open(corpus_dir + corpus_file, 'x')
# corpus.write("Now the file has more content!")
# corpus.close()
# !cat /content/dataset/corpus.txt

temp_dir = 'temp'
if not os.path.exists(temp_dir):
  os.mkdir(temp_dir)

gdrive = "GDrive/My Drive/corpus/segmented/"

file_list = os.listdir(gdrive)
# print(file_list)

for fname in file_list:
  if fname.endswith('.zip'):
    handleZip(corpus, gdrive + fname, temp_dir)
    continue

  tmp = open(gdrive + fname, 'r')
  corpus.write(tmp.read())
  tmp.close()

corpus.close()
# !cat /content/dataset/corpus.txt


**5. W2V trained by Gensim**

In [ ]:
from gensim.test.utils import datapath
from gensim import utils
from gensim.models.word2vec import PathLineSentences
import multiprocessing
import gensim.models

class MyCorpus:
  def __init__(self, corpus_path):
    self.cor_path = corpus_path

  """An interator that yields sentences (lists of str)."""

  def __iter__(self):
    print(self.cor_path)
    for line in PathLineSentences(self.cor_path):
      # print(line)
      # assume there's one document per line, tokens separated by whitespace
      yield line

sentences = MyCorpus(corpus_dir)
model = gensim.models.Word2Vec(
    sentences=sentences,
    min_count=1,
    size=300,
    window=15,
    workers=multiprocessing.cpu_count(),
    compute_loss=True,
    hs=0,
    sg=1,
    seed=42,
    iter=5
    )

**6. display the W2V**

In [83]:
for i, word in enumerate(model.wv.vocab):
  if i == 10:
    break
  print(word)
  print(model.wv[word])

滨湖
[ 3.20819095e-02  6.57614768e-02 -6.27478659e-02 -9.16682333e-02
  7.65243471e-02 -2.04404265e-01  8.27247724e-02 -1.37328997e-01
  2.60425471e-02  8.12623948e-02 -4.23536636e-02  8.82650837e-02
 -2.99170017e-02  1.39540628e-01 -1.91768575e-02  4.76948544e-02
 -1.42309159e-01 -5.75811081e-02 -7.48349121e-03  5.77619337e-02
  2.61928439e-02  2.61370800e-02  4.78960276e-02  4.71839157e-04
 -5.68802804e-02 -1.06146839e-02 -1.91162676e-02  2.34614283e-01
 -6.93610832e-02 -9.69257858e-03 -1.25648662e-01 -2.24678710e-01
 -1.17678016e-01 -2.02483311e-02 -1.93074673e-01  2.05795150e-02
  1.31881997e-01  4.74181250e-02  1.04476847e-02  5.34051172e-02
 -3.77877355e-02  1.24230180e-02  1.00143254e-02 -1.77589774e-01
 -1.71913523e-02  2.53524810e-01  2.32084468e-01  3.98828536e-02
  6.50395304e-02  1.21298373e-01  8.70005190e-02 -4.57561046e-01
  1.64653972e-01 -2.65345573e-02  1.92355702e-03  1.38206571e-01
 -4.98000383e-02 -1.00276275e-02 -1.29996151e-01  8.53244141e-02
  1.04978107e-01  6.15

**7. Store the model file**

In [85]:
models_dir = dataset_dir + '/models'
if not os.path.exists(models_dir):
  os.mkdir(models_dir)
model_file = '/model.txt'
# remove previous file instance
if os.path.exists(models_dir + model_file):
  os.remove(models_dir + model_file)

model.save(models_dir + model_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


**8. Demo**

In [84]:
print('%r\t%r\t%.2f' % ("维蒂格豪森", "迪欣根", model.similarity("维蒂格豪森", "迪欣根")))

print('%r\t%r\t%.2f' % ("中華民國", "迪欣根", model.similarity("中華民國", "迪欣根")))

'维蒂格豪森'	'迪欣根'	0.95
'中華民國'	'迪欣根'	0.33


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
